---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(5)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [2]:
# Importing regular expressions and looking for the pattern 'mm/dd/yy' or 'mm/dd/yyyy' whith '/' or '-'

import re
text_df = pd.DataFrame(df,columns=['OriginalText'])
text_df['p_date'] = text_df['OriginalText'].str.extract(r'(\d?\d[/\//-]\d?\d[/\//-]\d{2,4})')
text_df['date'] = pd.to_datetime(text_df['p_date'], errors='ignore')
text_df.head(5)     

,OriginalText,p_date,date
0,03/25/93 Total time of visit (in minutes):\n,03/25/93,1993-03-25
1,6/18/85 Primary Care Doctor:\n,6/18/85,1985-06-18
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71,1971-07-08
3,7 on 9/27/75 Audit C Score Current:\n,9/27/75,1975-09-27
4,2/6/96 sleep studyPain Treatment Pain Level (N...,2/6/96,1996-02-06


In [3]:
# Those rows whose date was identified goes to 'treated' dataframe and the others to 'text_df2', on whitch we're going to look for other patterns

treated = text_df[~text_df['date'].isna()]
text_df2 = text_df[text_df['date'].isna()]

un = text_df2.count()[0]
ma = treated.count()[0]
per = (ma/(un+ma)) * 100


print(f'Matched: {ma} rows,({per}%)')

Matched: 125 rows,(25.0%)


In [4]:
# Looking for the patthern 'dd mmm(name) yyyy'

def months_str(a):    
    a = a.lower()
    date = re.findall(r'\d{1,2} (?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) \d{2,4}',a)
    return str(date)[2:-2]

text_df2['p_date'] = text_df2['OriginalText'].apply(lambda x:months_str(x))
text_df2['date'] = pd.to_datetime(text_df2['p_date'], errors='ignore')
text_df2

,OriginalText,p_date,date
125,s The patient is a 44 year old married Caucasi...,24 jan 2001,2001-01-24
126,.10 Sep 2004 - Intake at EEC for IOP but did n...,10 sep 2004,2004-09-10
127,see above and APS eval of 26 May 1982 Social H...,26 may 1982,1982-05-26
128,Tbooked for intake appointment at Sierra Vista...,,NaT
129,06 May 1972 SOS-10 Total Score:\n,06 may 1972,1972-05-06
...,...,...,...
495,1979 Family Psych History: Family History of S...,,NaT
496,therapist and friend died in ~2006 Parental/Ca...,,NaT
497,2008 partial thyroidectomy\n,,NaT
498,sPt describes a history of sexual abuse as a c...,,NaT


In [5]:
treated = treated.append(text_df2[~text_df2['date'].isna()])

text_df3 = text_df2[text_df2['date'].isna()]

un = text_df3.count()[0]
ma = treated.count()[0]
per = (ma/(un+ma)) * 100
print(f'Matched: {ma} rows,({per}%)')

text_df3.head()



Matched: 185 rows,(37.0%)


,OriginalText,p_date,date
128,Tbooked for intake appointment at Sierra Vista...,,NaT
133,h missed intake office visit on 28 June 1994 a...,,NaT
136,11 February 1985 CPT Code: 90801 - Psychiatric...,,NaT
150,12 March 1980 SOS-10 Total Score:\n,,NaT
151,22 June 1990 Medical History:\n,,NaT


In [6]:
# Since we didin't consider those rows with the whole month number, lets do it replace those for the abreviation now and repeat the process

def abv_months(a):

    months_lookup = {
    'january':'jan',
    'february':'feb',
    'march':'mar',
    'april':'apr',
    'may':'may',
    'june':'jun',
    'july':'jul',
    'august':'aug',
    'september':'sep',
    'october':'oct',
    'november':'nov',
    'december':'dec',
    }

    for key in months_lookup.keys():
        a = a.lower().replace(key, months_lookup[key])
    return a

text_df3['OriginalText'] = text_df3['OriginalText'].apply(lambda x:abv_months(x))
text_df3['p_date'] = text_df3['OriginalText'].apply(lambda x:months_str(x))
text_df3['date'] = pd.to_datetime(text_df3['p_date'], errors='ignore')
text_df3

,OriginalText,p_date,date
128,tbooked for intake appointment at sierra vista...,28 jun 2002,2002-06-28
133,h missed intake office visit on 28 jun 1994 at...,28 jun 1994,1994-06-28
136,11 feb 1985 cpt code: 90801 - psychiatric diag...,11 feb 1985,1985-02-11
150,12 mar 1980 sos-10 total score:\n,12 mar 1980,1980-03-12
151,22 jun 1990 medical history:\n,22 jun 1990,1990-06-22
...,...,...,...
495,1979 family psych history: family history of s...,,NaT
496,therapist and friend died in ~2006 parental/ca...,,NaT
497,2008 partial thyroidectomy\n,,NaT
498,spt describes a history of sexual abuse as a c...,,NaT


In [7]:
treated = treated.append(text_df3[~text_df2['date'].isna()])
text_df4 = text_df3[text_df3['date'].isna()]

un = text_df4.count()[0]
ma = treated.count()[0]
per = (ma/(un+ma)) * 100
print(f'Matched: {ma} rows,({per}%)')


text_df4.head()

Matched: 185 rows,(37.67820773930753%)


/tmp/ipykernel_20229/3004337168.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  treated = treated.append(text_df3[~text_df2['date'].isna()])


,OriginalText,p_date,date
194,"apr 11, 1990 cpt code: 90791: no medical servi...",,NaT
195,"mri may 30, 2001 empty sella but no problems w...",,NaT
196,".feb 18, 1994: made a phone call to mom and mo...",,NaT
197,"brother died feb 18, 1981 parental/caregiver o...",,NaT
198,none; but currently has appt with new hjh pcp ...,,NaT


In [8]:
# Aparently, the pattern like 'apr 11, 1990' is relevant here, lets try to identify it

t = ['apr 11, 1990 cpt code: 90791: no medical services\n','mri may 30, 2001 empty sella but no problems with endocrine functionpertinent medical review of systems constitutional:\n','mar. 10, 1976 cpt code: 90791: no medical services\n']

for tx in t:
    print(re.findall(r'(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\.?\s\d{1,2}\,\s\d{2,4}',tx))

['apr 11, 1990']
['may 30, 2001']
['mar. 10, 1976']


In [9]:
# Now, put it in a lambda function and cointinue the matching process

def months_str_2(a):
    
    a = a.lower()

    date = re.findall(r'(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\.?\s\d{1,2}\,?\s\d{2,4}',a)

    return str(date)[2:-2]


text_df4['p_date'] = text_df4['OriginalText'].apply(lambda x:months_str_2(x))
text_df4['date'] = pd.to_datetime(text_df4['p_date'], errors='ignore')
text_df4

,OriginalText,p_date,date
194,"apr 11, 1990 cpt code: 90791: no medical servi...","apr 11, 1990",1990-04-11
195,"mri may 30, 2001 empty sella but no problems w...","may 30, 2001",2001-05-30
196,".feb 18, 1994: made a phone call to mom and mo...","feb 18, 1994",1994-02-18
197,"brother died feb 18, 1981 parental/caregiver o...","feb 18, 1981",1981-02-18
198,none; but currently has appt with new hjh pcp ...,"oct. 11, 2013",2013-10-11
...,...,...,...
495,1979 family psych history: family history of s...,,NaT
496,therapist and friend died in ~2006 parental/ca...,,NaT
497,2008 partial thyroidectomy\n,,NaT
498,spt describes a history of sexual abuse as a c...,,NaT


In [10]:
treated = treated.append(text_df4[~text_df4['date'].isna()])
text_df5 = text_df4[text_df4['date'].isna()]

un = text_df5.count()[0]
ma = treated.count()[0]
per = (ma/(un+ma)) * 100
print(f'Matched: {ma} rows,({per}%)')

text_df5.head(10)

Matched: 219 rows,(44.602851323828915%)


,OriginalText,p_date,date
228,"s 20 yo m carries dx of bpad, presents for psy...",,NaT
229,t allergies sulfa (sulfonamide antibiotics) - ...,,NaT
230,b/r walnut ridge. raised with sister and paren...,,NaT
231,50 yo dwf with a history of alcohol use disord...,,NaT
232,")htn, hypercholesterolemia, dm, sleep apnea,, ...",,NaT
233,"dr. gloria english, who conducted an initial c...",,NaT
234,. patient presents with gad flareup in jul 200...,,NaT
235,poct 2015 - admitted to gray clinic for depres...,,NaT
236,pt does not use marijuana currently. her last ...,,NaT
237,"safter evicted in feb 1976, hospitalized at pe...",,NaT


In [11]:
# Aparently, the pattern like 'sep 1985' is relevant here, lets try to identify it and repeat the process

t = ['20 yo m carries dx of bpad, presents for psychopharm consult. moved to independence area for school as of sep 1985',' allergies sulfa (sulfonamide antibiotics) - renal toxicity : pt developed acute interstitial nephritis on bactrim (jun 2011)\n']

for tx in t:
    print(re.findall(r'(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\s\d{2,4}',tx))

['sep 1985']
['jun 2011']


In [12]:
# Another lambda function and cointinue the matching process

def months_str_3(a):
    
    a = a.lower()

    date = re.findall(r'(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\,?\s\d{2,4}',a)

    return str(date)[2:-2]


text_df5['OriginalText'] = text_df5['OriginalText'].apply(lambda x:abv_months(x))
text_df5['p_date'] = text_df5['OriginalText'].apply(lambda x:months_str_3(x))
text_df5['date'] = pd.to_datetime(text_df5['p_date'], errors='ignore')
text_df5.loc[298]

OriginalText    hh, janaury 1993\n
p_date                            
date                           NaT
Name: 298, dtype: object

In [13]:
treated = treated.append(text_df5[~text_df5['date'].isna()])
text_df6 = text_df5[text_df5['date'].isna()]

un = text_df6.count()[0]
ma = treated.count()[0]
per = (ma/(un+ma)) * 100
print(f'Matched: {ma} rows,({per}%)')

text_df6.head(10)

Matched: 332 rows,(67.61710794297352%)


,OriginalText,p_date,date
298,"hh, janaury 1993\n",,NaT
313,nloss of father to cardiac event in decemeber ...,,NaT
343,6/1998 primary care doctor:\n,,NaT
344,"s 52 y/o mwm h/o chronic depression, anxiety, ...",,NaT
345,10/1973 hx of brain injury: yes\n,,NaT
346,9/2005 primary care doctor:\n,,NaT
347,s 03/1980 positive ppd: treated with inh for 6...,,NaT
348,12/2005 family psych history: family history o...,,NaT
349,5/1987 primary care doctor:\n,,NaT
350,5/2004 primary care doctor:\n,,NaT


In [14]:
# Looking for the patthern 'm(or mm) /( or -) yyyy '

text_df6['p_date'] = text_df6['OriginalText'].str.extract(r'(\d?\d[/\//-]\d{2,4})')
text_df6['date'] = pd.to_datetime(text_df6['p_date'], errors='ignore')
text_df6.head()


,OriginalText,p_date,date
298,"hh, janaury 1993\n",NaN,NaT
313,nloss of father to cardiac event in decemeber ...,NaN,NaT
343,6/1998 primary care doctor:\n,6/1998,1998-06-01
344,"s 52 y/o mwm h/o chronic depression, anxiety, ...",6/2005,2005-06-01
345,10/1973 hx of brain injury: yes\n,10/1973,1973-10-01


In [15]:
treated = treated.append(text_df6[~text_df6['date'].isna()])
text_df7 = text_df6[text_df6['date'].isna()]

un = text_df7.count()[0]
ma = treated.count()[0]
per = (ma/(un+ma)) * 100
print(f'Matched: {ma} rows,({per}%)')

text_df7.head()

Matched: 444 rows,(90.4276985743381%)


,OriginalText,p_date,date
298,"hh, janaury 1993\n",NaN,NaT
313,nloss of father to cardiac event in decemeber ...,NaN,NaT
455,shemmorage caused by probe in 1984 medical his...,NaN,NaT
456,shas been at myh since his treaters in ne reti...,NaN,NaT
457,pt joined army reserves in 2001 and has 3 year...,NaN,NaT


In [16]:
# With 90% of the data already converted to the desired output, its time to look for the more general pattern, just the year

text_df7['p_date'] = text_df6['OriginalText'].str.extract(r'(\d{4})')
text_df7['date'] = pd.to_datetime(text_df7['p_date'], errors='ignore')
text_df7.head()

,OriginalText,p_date,date
298,"hh, janaury 1993\n",1993,1993-01-01
313,nloss of father to cardiac event in decemeber ...,1978,1978-01-01
455,shemmorage caused by probe in 1984 medical his...,1984,1984-01-01
456,shas been at myh since his treaters in ne reti...,2000,2000-01-01
457,pt joined army reserves in 2001 and has 3 year...,2001,2001-01-01


In [17]:
treated = treated.append(text_df7[~text_df7['date'].isna()])

ma = treated.count()[0]

print(f'Matched: {ma} rows')

Matched: 491 rows,(98.2%)


In [21]:
#Finaly, there's some rows with unmached data whith the patter like 'day month_name year'. Whith that, we sort the data and finish our task

r = []
for j in range(500):
    if j in treated.index:
        pass
    else:
        r.append(j)treated.sort_values(by='date')

def months_str_4(a):
    a = a.lower()
    date = re.findall(r'd?\d\s(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\,?\s\d{2,4}',a)
    return str(date)[2:-2]




text_df8['OriginalText'] = text_df8['OriginalText'].apply(lambda x:abv_months(x))
text_df8['p_date'] = text_df8['OriginalText'].apply(lambda x:months_str_4(x))
text_df8['date'] = pd.to_datetime(text_df8['p_date'], errors='ignore')
text_df8.head()

treated = treated.append(text_df8[~text_df8['date'].isna()])

treated.count()

OriginalText    500
p_date          500
date            500
dtype: int64

In [42]:
#Aparently, we got it right.
treated.sort_values(by='date')

,OriginalText,p_date,date
9,(4/10/71)Score-1Audit C Score Current:\n,4/10/71,1971-04-10
84,5/18/71 Total time of visit (in minutes):\n,5/18/71,1971-05-18
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71,1971-07-08
53,7/11/71 SOS-10 Total Score:\n,7/11/71,1971-07-11
28,9/12/71 [report_end]\n,9/12/71,1971-09-12
...,...,...,...
231,50 yo dwf with a history of alcohol use disord...,may 2016,2016-05-01
141,30 May 2016 SOS-10 Total Score:\n,30 may 2016,2016-05-30
186,13 Oct 2016 Primary Care Doctor:\n,13 oct 2016,2016-10-13
161,19 Oct 2016 Communication with referring physi...,19 oct 2016,2016-10-19


In [38]:
def date_sorter():
    
    pd.options.mode.chained_assignment = None  # default='warn'
    
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)
        df = pd.Series(doc)
    
    import re
    text_df = pd.DataFrame(df,columns=['OriginalText'])
    text_df['p_date'] = text_df['OriginalText'].str.extract(r'(\d?\d[/\//-]\d?\d[/\//-]\d{2,4})')
    text_df['date'] = pd.to_datetime(text_df['p_date'], errors='ignore')
    
    treated = text_df[~text_df['date'].isnull()]
    text_df2 = text_df[text_df['date'].isnull()]
    
    def months_str(a):    
        a = a.lower()
        date = re.findall(r'\d{1,2} (?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) \d{2,4}',a)
        return str(date)[2:-2]

    text_df2['p_date'] = text_df2['OriginalText'].apply(lambda x:months_str(x))
    text_df2['date'] = pd.to_datetime(text_df2['p_date'], errors='ignore')
    
    treated = treated.append(text_df2[~text_df2['date'].isnull()])
    text_df3 = text_df2[text_df2['date'].isnull()]
    
    def abv_months(a):
        months_lookup = {
        'january':'jan',
        'february':'feb',
        'march':'mar',
        'april':'apr',
        'may':'may',
        'june':'jun',
        'july':'jul',
        'august':'aug',
        'september':'sep',
        'october':'oct',
        'november':'nov',
        'december':'dec',
        }
        for key in months_lookup.keys():
            a = a.lower().replace(key, months_lookup[key])
        return a

    text_df3['OriginalText'] = text_df3['OriginalText'].apply(lambda x:abv_months(x))
    text_df3['p_date'] = text_df3['OriginalText'].apply(lambda x:months_str(x))
    text_df3['date'] = pd.to_datetime(text_df3['p_date'], errors='ignore')
    
    treated = treated.append(text_df3[~text_df2['date'].isnull()])
    text_df4 = text_df3[text_df3['date'].isnull()]
    
    def months_str_2(a):
        a = a.lower()
        date = re.findall(r'(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\.?\s\d{1,2}\,?\s\d{2,4}',a)
        return str(date)[2:-2]

    text_df4['p_date'] = text_df4['OriginalText'].apply(lambda x:months_str_2(x))
    text_df4['date'] = pd.to_datetime(text_df4['p_date'], errors='ignore')
    
    treated = treated.append(text_df4[~text_df4['date'].isnull()])
    text_df5 = text_df4[text_df4['date'].isnull()]
    
    def months_str_3(a):
        a = a.lower()
        date = re.findall(r'(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\,?\s\d{2,4}',a)
        return str(date)[2:-2]


    text_df5['OriginalText'] = text_df5['OriginalText'].apply(lambda x:abv_months(x))
    text_df5['p_date'] = text_df5['OriginalText'].apply(lambda x:months_str_3(x))
    text_df5['date'] = pd.to_datetime(text_df5['p_date'], errors='ignore')
    
    treated = treated.append(text_df5[~text_df5['date'].isnull()])
    text_df6 = text_df5[text_df5['date'].isnull()]
    
    text_df6['p_date'] = text_df6['OriginalText'].str.extract(r'(\d?\d[/\//-]\d{2,4})')
    text_df6['date'] = pd.to_datetime(text_df6['p_date'], errors='ignore')
    
    treated = treated.append(text_df6[~text_df6['date'].isnull()])
    text_df7 = text_df6[text_df6['date'].isnull()]
    
    text_df7['p_date'] = text_df6['OriginalText'].str.extract(r'(\d{4})')
    text_df7['date'] = pd.to_datetime(text_df7['p_date'], errors='ignore')
    
    treated = treated.append(text_df7[~text_df7['date'].isnull()])
    
    r = []
    
    for j in range(500):
        if j in treated.index:
            pass
        else:
            r.append(j)

    text_df8 = text_df.iloc[r,:]

    def months_str_4(a):
        a = a.lower()
        date = re.findall(r'd?\d\s(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\,?\s\d{2,4}',a)
        return str(date)[2:-2]




    text_df8['OriginalText'] = text_df8['OriginalText'].apply(lambda x:abv_months(x))
    text_df8['p_date'] = text_df8['OriginalText'].apply(lambda x:months_str_4(x))
    text_df8['date'] = pd.to_datetime(text_df8['p_date'], errors='ignore')
    text_df8.head()

    treated = treated.append(text_df8[~text_df8['date'].isnull()])

    
    
    return pd.Series(treated.sort_values(by='date').index)# Your answer here

In [37]:
pd.Series(date_sorter().index)

/tmp/ipykernel_20229/359132400.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  treated = treated.append(text_df3[~text_df2['date'].isnull()])


0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64